# EP04: 가성비 좋은 스팟 인스턴스로 비용 아끼기

오프닝: 간단한 본인 소개

Q: 이번에 살펴볼 내용은 어떤 내용인가요?

A: 이번 에피소드에서는 스팟 인스턴스를 이용하여 원하는 가격 범위 안에서 계획적으로 쿠버네티스 클러스터를 구축하고 운영하는 방법을 소개합니다.

Q: 스팟 인스턴스에 대해 좀 더 자세히 이야기해주실 수 있나요?

A: 스팟 인스턴스는 애저 데이터센터 내에 할당되지 않은 여유있는 가상 컴퓨터를 원하는 가격에 저렴하게 사용하고, 애저 데이터센터에서 다시 필요로 할 때에는 반환하는 사용 방식입니다.

![Spot VM](EP04.png)

Q: 그런데 쿠버네티스 노드에서 사용한다면 어떻게 쓸 수 있나요?

A: 미리 구성된 쿠버네티스 노드 이미지를 사용하여 클러스터에 가입할 수만 있다면, 쿠버네티스에 컨테이너를 띄우는 것은 아무런 제약 사항이 없습니다.

그래서 복잡하게 미리 인스턴스를 구성하는데 노력이나 시간을 들이지 않고 스팟 인스턴스를 확보하자마자 빠르게 쿠버네티스용 노드로 쓸 수 있습니다.

A: 반대로 다시 인스턴스를 내주어야 할 때에는 모든 외부 데이터를 애저 관리 디스크나 블랍 스토리지 등에 따로 저장할 수 있으므로 안전하고 신속하게 노드와 노드 위의 컨테이너들을 지울 수 있습니다.

약 30초 정도를 앞두고 인스턴스를 비워주어야 한다는 것을 애저에서 알려주므로 쿠버네티스 클러스터는 여기에 맞추어 모든 컨테이너들을 재배치하므로 서비스 중단 현상도 최소화할 수 있습니다.

https://bit.ly/rkttu-use-spot-vm-in-azure

Q: 스팟 인스턴스임을 지정할 수 있는 방법이 있나요?

A: 앞의 에피소드에서 본 것처럼 가상 머신 스케일 셋을 사용합니다. 다른 점은 스팟 인스턴스를 사용할 것임을 명시하기만 하면 됩니다.

지금부터 자세한 내용을 살펴보도록 하겠습니다.

## 애저 로그인 상태 확인

우선 애저 커맨드 라인 도구의 로그인 상태를 먼저 확인합니다. 로그인 상태가 아니면 다른 터미널에서 `az login` 명령을 실행합니다. 설치되어있지 않다면 OS 버전 별로 애저 커맨드 라인 도구를 설치하고 준비합니다.

In [ ]:
az account list

## AKS 엔진 설치 상태 확인

아래 명령어를 실행하여 `aks-engine`이 설치되었는지 확인합니다. 설치되어있지 않다면 OS 버전 별로 각각 설치 가이드를 따라 설치를 진행합니다.

In [ ]:
aks-engine version

## 서비스 주체 생성

쿠버네티스 클러스터를 만든 후에는 클러스터 수준에서 스스로 애저 리소스들을 제어할 수 있어야 하므로, 서비스 주체 계정을 만들어 계속 해당 계정을 사용하도록 설정합니다.

In [ ]:
$SubscriptionData = (az account list -o json) | ConvertFrom-Json -Depth 16
$SubscriptionId = $SubscriptionData[0].id
$SvcPrincipal = (az ad sp create-for-rbac `
  -n="AzurePlaylist" --role="Contributor" `
  --scopes="/subscriptions/$SubscriptionId" `
  -o json) | ConvertFrom-Json -Depth 16

$PrincipalId = $SvcPrincipal.AppId
$PrincipalPwd = $SvcPrincipal.Password

"Your App ID is $PrincipalId"

## SSH 키 준비하기

아래 명령어를 실행하여 SSH 키의 공개 키를 준비합니다. 설치되어있지 않다면 SSH 키 쌍을 만듭니다.

In [ ]:
$PublicKeyValue = Get-Content -Path "$HOME\.ssh\id_rsa.pub"

## API 모델 준비하기

이제 API 모델을 준비할 차례입니다. AKS 엔진은 코드 기반 인프라 관리 방식을 구현하기 위하여 JSON API 모델을 작성하면, 여기에 맞추어 ARM 템플릿을 만들어줍니다. 이것을 애저 명령줄 도구를 통해 배포하면 손쉽게 쿠버네티스 클러스터를 생성할 수 있습니다.

In [ ]:
$ApiModel = @"
{
  'apiVersion': 'vlabs',
  'properties': {
    'orchestratorProfile': {
      'orchestratorType': 'Kubernetes',
      'kubernetesConfig': {
        'useManagedIdentity': true
      }
    },
    'servicePrincipalProfile': {
      'clientId': '',
      'secret': ''
    },
    'masterProfile': {
      'count': 1,
      'dnsPrefix': '',
      'vmSize': 'Standard_D2_v3'
    },
    'agentPoolProfiles': [
      {
        'name': 'agentpool',
        'count': 1,
        'vmSize': 'Standard_D2_v3',
        'availabilityProfile': 'VirtualMachineScaleSets',
        'scaleSetPriority': 'Spot',
        'scaleSetEvictionPolicy': 'Delete',
        'spotMaxPrice': 88,
        'storageProfile': 'ManagedDisks'
      }
    ],
    'linuxProfile': {
      'adminUsername': 'azureuser',
      'ssh': {
        'publicKeys': [
          {
            'keyData': ''
          }
        ]
      }
    }
  }
}
"@ | ConvertFrom-Json -Depth 16

## API 모델 내용 변경하기

나중에 리눅스 마스터 노드에 접속할 수 있도록 현재 컴퓨터에서 사용하는 SSH 공개 키를 배포할 때 같이 등록합니다. 그리고 서비스 주체를 가리키는 아이디 값과 고유 비밀 키 값을 API 모델에 같이 지정합니다.

In [ ]:
$ApiModel.properties.linuxProfile.ssh.publicKeys[0].keyData = "$PublicKeyValue"
$ApiModel.properties.servicePrincipalProfile.clientId = "$PrincipalId"
$ApiModel.properties.servicePrincipalProfile.secret = "$PrincipalPwd"

In [ ]:
$ApiModel | ConvertTo-Json -Depth 16 | Out-File -Encoding utf8 -Path 'ep04.json' -Force

dir 'ep04*'

## API 모델을 ARM 템플릿으로 변환하고 배포하기

다음의 명령어를 실행하여 API 모델을 ARM 템플릿으로 변환하고 곧바로 애저에 배포하도록 하겠습니다. 이 명령어는 최초에 한 번만 실행하고, 그 이후에는 `ep03_output` 디렉터리 안에 있는 파일들을 이용하여 클러스터를 관리하도록 합니다.

In [ ]:
$DnsPrefix = 'rkttuep04'
$ResourceGroup = 'rkttuep04'
$Location = 'westus2'

In [ ]:
aks-engine deploy --dns-prefix "$DnsPrefix" `
    --resource-group "$ResourceGroup" `
    --location "$Location" `
    --api-model "ep04.json" `
    --auto-suffix `
    --output-directory "ep04_output" `
    --auth-method cli

## kubectl 실행해보기

클러스터가 잘 만들어졌는지 확인해보기 위하여 다음과 같이 `KUBECONFIG` 환경 변수를 설정하여 클러스터에 접근하고, 노드와 구성 상태를 확인합니다.

In [ ]:
$env:KUBECONFIG="ep04_output/kubeconfig/kubeconfig.$Location.json"
kubectl get nodes -o wide

## 리소스 그룹 정리

테스트를 위해 생성한 리소스 그룹을 제거하고 모든 리소스를 일괄 정리합니다.

In [ ]:
az group delete --name $ResourceGroup --yes --no-wait

클로징: 이 동영상을 보고 애저를 써봐야겠다고 생각하셨나요? 애저 계정은 누구나 무료로 만들 수 있습니다. 지금 바로 시작해보세요. (가능하다면 할인/무료 쿠폰을 포함한 가입 링크 제공) 다음 에피소드에서는 클라우드 셸을 사용할 수 있는 다양한 방법을 알아보겠습니다.